In [65]:
%load_ext autoreload
%autoreload 2

import mycoco

mycoco.setmode('train')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading annotations into memory...
Done (t=13.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.91s)
creating index...
index created!


In [66]:
all_ids = mycoco.query([['']])

In [67]:
import pickle

with open('./data/tokenizer10000.pickle', 'rb') as f:
    tokenizer = pickle.load(f)
    
tokenizer

In [73]:
from keras.models import load_model

encoder = load_model('./models/encoder.model.hdf5')
encoder.compile()

/usr/local/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


TypeError: compile() missing 1 required positional argument: 'optimizer'

In [69]:
cap_examples = mycoco.iter_captions_examples(all_ids, tokenizer, encoder)

In [70]:
next(cap_examples)[1]

/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)],
 array([[0.46396238, 0.23454374, 0.        , ..., 0.08909912, 0.31195068,
         0.29884964]], dtype=float32)]

In [71]:
from keras.models import Model
from keras.layers import Dense, LSTM, Embedding, Input

input_length = 5
embed_size = 50
vocab_size = 10000
img_vec_size = 5000

inputs = Input(shape=(input_length,))
embed = Embedding(vocab_size, embed_size, input_length=input_length)(inputs)
lstm = LSTM(50, dropout=0.1)(embed)
# Word prediction softmax
word_pred = Dense(vocab_size, activation='softmax', name='word_prediction')(lstm)
image_vec_preds = Dense(img_vec_size, activation = 'sigmoid', name='image_vec_prediction')(lstm)

# This creates a model that includes
# the Input layer and two Dense layers outputs
model = Model(inputs=inputs, outputs=[word_pred, image_vec_preds])

model.compile(optimizer='adam',
        loss={
            'word_prediction': 'categorical_crossentropy',
            'image_vec_prediction': 'binary_crossentropy'
        },
        metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 5, 50)        500000      input_6[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, 50)           20200       embedding_6[0][0]                
__________________________________________________________________________________________________
word_prediction (Dense)         (None, 10000)        510000      lstm_6[0][0]                     
__________________________________________________________________________________________________
image_vec_

In [72]:
model.fit_generator(cap_examples, steps_per_epoch=100, epochs=30)

Epoch 1/30
 41/100 [===========>..................] - ETA: 2s - loss: 9.8874 - word_prediction_loss: 9.2016 - image_vec_prediction_loss: 0.6858 - word_prediction_acc: 0.0244 - image_vec_prediction_acc: 0.2743

/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


100/100 [==============================] - 3s 30ms/step - loss: 9.5729 - word_prediction_loss: 8.9611 - image_vec_prediction_loss: 0.6118 - word_prediction_acc: 0.0500 - image_vec_prediction_acc: 0.3154
Epoch 2/30
100/100 [==============================] - 2s 19ms/step - loss: 8.0058 - word_prediction_loss: 7.7042 - image_vec_prediction_loss: 0.3016 - word_prediction_acc: 0.0500 - image_vec_prediction_acc: 0.3550
Epoch 3/30
100/100 [==============================] - 2s 19ms/step - loss: 7.9948 - word_prediction_loss: 7.3949 - image_vec_prediction_loss: 0.5998 - word_prediction_acc: 0.0800 - image_vec_prediction_acc: 0.2466
Epoch 4/30
100/100 [==============================] - 2s 19ms/step - loss: 7.3477 - word_prediction_loss: 6.8338 - image_vec_prediction_loss: 0.5139 - word_prediction_acc: 0.0500 - image_vec_prediction_acc: 0.2932
Epoch 5/30
100/100 [==============================] - 2s 19ms/step - loss: 7.3283 - word_prediction_loss: 6.8762 - image_vec_prediction_loss: 0.4521 - word